In [1]:
%pip install selenium


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

driver.get('https://kampusmerdeka.kemdikbud.go.id/program/magang/browse')  

search_input = driver.find_element(By.CSS_SELECTOR, '.inputBox-0-0-265')

search_input.send_keys('Mobile')

search_input.send_keys(Keys.RETURN)

SCROLL_PAUSE_TIME = 2
time.sleep(SCROLL_PAUSE_TIME)

scrollable_div = driver.find_element(By.CSS_SELECTOR, '.scroll-0-0-855')

last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)

while True:
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)

    time.sleep(SCROLL_PAUSE_TIME)

    new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    if new_height == last_height:
        break
    last_height = new_height

cards = driver.find_elements(By.CSS_SELECTOR, '.description-0-0-811')

data = []
for card in cards:
    try:
        title = card.find_element(By.CSS_SELECTOR, 'p.heading-5-0-0-77').text
        company = card.find_element(By.CSS_SELECTOR, 'p.text-0-0-56.sans-0-0-70.tiny-0-0-72.ellipsis-0-0-59.pad-0-0-1221').text
        daerah = card.find_element(By.CSS_SELECTOR, 'p.text-0-0-56.sans-0-0-70.tiny-0-0-72.pad-0-0-1221 span.text-0-0-56.sans-0-0-70').text
        data.append([title, company, daerah])
    except NoSuchElementException:
        continue

with open('Mobile.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "Company", "Daerah"])
    writer.writerows(data)

driver.quit()